# Fine Tuning a Pretrained Model
* Part 3: Train a model without using the `Trainer` class. Instead, we use PyTorch + `accelerate`

This Jupyter Notebook was not fully run due to the execution of the `Trainer` class being much faster using GPUs. A python equivalent and a slurm job have been included in the `runs` folder for the purpose of running the code displayed below.

In [67]:
import evaluate
import torch
import torch.nn.functional as F

from datasets import load_dataset
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    get_scheduler
)

In [30]:
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

Found cached dataset glue (/n/fs/nlp-jy1682/hf_datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

### Prepare for Training
* Define a dataloader to iterate over batches
* Apply postprocessing to `tokenized_datasets`
    * Remove columns corresponding to unexpected values
    * Rename column `label` -> `labels`
    * Set dataset format to Pytorch, not lists

In [31]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names # Check that this is columns model will accept

Loading cached processed dataset at /n/fs/nlp-jy1682/hf_datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-2286bfeed35ec98c.arrow
Loading cached processed dataset at /n/fs/nlp-jy1682/hf_datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-d2871c4b8b165e62.arrow
Loading cached processed dataset at /n/fs/nlp-jy1682/hf_datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-e4e2a9b2438b2f62.arrow


['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [33]:
# Create dataloader objects
train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [34]:
# Verify data processing executed correctly
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 76]),
 'token_type_ids': torch.Size([8, 76]),
 'attention_mask': torch.Size([8, 76])}

### Model

In [36]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [53]:
# Check that model will accept data
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape) # 2 logits

tensor(0.6039, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [54]:
F.softmax(outputs.logits, dim=-1)

tensor([[0.1351, 0.8649],
        [0.1408, 0.8592],
        [0.1421, 0.8579],
        [0.1401, 0.8599],
        [0.1371, 0.8629],
        [0.1393, 0.8607],
        [0.1379, 0.8621],
        [0.1382, 0.8618]], grad_fn=<SoftmaxBackward0>)

### Training Components

In [59]:
# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

In [60]:
# Learning Rate Scheduler
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

1377


### Training Loop

In [62]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cpu')

In [ ]:
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

### Evaluation Loop

In [ ]:
metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

The example with `accelerate` is included in the `runs` folder